<a href="https://colab.research.google.com/github/yun-aha/GSTEC/blob/main/210709.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. 빅데이터 분석

### SQLite 데이터 베이스와 Pandas 데이터프레임

In [ ]:
import sqlite3
import pandas as pd
import os

In [ ]:
from google.colab import drive
# 구글 드라이브 마운트.
drive.mount('/content/drive')  # 절차를 따라서 한다

Mounted at /content/drive


In [ ]:
# 현 폴더의 위치를 보여준다.
!pwd        # Linux

/content


In [ ]:
# 작업 폴더로 이동한다.
# 데이터 파일은 이미 올려져 있어야 한다.
os.chdir("/content/drive/MyDrive/광인사/01_SQL과 데이터 베이스_배포/")
# %cd /content/drive/MyDrive/광인사/01_SQL과 데이터 베이스_배포/        #디렉토리 경로이동 

In [ ]:
# 현 폴더의 위치를 보여준다.
!pwd

/content/drive/MyDrive/광인사/01_SQL과 데이터 베이스_배포


In [ ]:
# 현 폴더의 내용을 보여준다.
!ls

'0101 - SQL과 데이터 베이스.pdf'   scripts
 data				   sqlitestudio-3.3.3
 database			  '빅데이터분석 - 커리큘럼.txt'
 notebook


#### 1). 데이터프레임의 내용을 데이터베이스로 옮겨본다: 
#### 데이터프레임을 가져온다:

In [ ]:
os.chdir("./data")
df = pd.read_csv("data_studentlist_en.csv",header="infer")

In [ ]:
nrows = df.shape[0]
my_header = df.columns
df.head(3)

,name,gender,age,grade,absence,bloodtype,height,weight
0,Jared Diamond,M,23,3,Y,O,165.3,68.2
1,Sarah O'Donnel,F,22,2,N,AB,170.1,53.0
2,Brian Martin,M,24,4,N,B,175.0,80.1


In [ ]:
df.describe()

,age,grade,height,weight
count,17.00,17.00,17.00,17.00
mean,22.35,2.29,170.04,61.06
std,1.17,1.10,7.85,11.85
min,20.00,1.00,155.20,45.20
25%,22.00,1.00,165.30,53.00
50%,22.00,2.00,169.20,61.30
75%,23.00,3.00,176.10,68.20
max,24.00,4.00,182.10,85.70


In [ ]:
pd.set_option('precision',2)    #소수점 이하 2자리까지만 보여준다##pandas자체 설정 바꾸기

In [ ]:
df.describe()       #다양한 통계량을 요약해준다.

,age,grade,height,weight
count,17.00,17.00,17.00,17.00
mean,22.35,2.29,170.04,61.06
std,1.17,1.10,7.85,11.85
min,20.00,1.00,155.20,45.20
25%,22.00,1.00,165.30,53.00
50%,22.00,2.00,169.20,61.30
75%,23.00,3.00,176.10,68.20
max,24.00,4.00,182.10,85.70


In [ ]:
# 구조를 본다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       17 non-null     object 
 1   gender     17 non-null     object 
 2   age        17 non-null     int64  
 3   grade      17 non-null     int64  
 4   absence    17 non-null     object 
 5   bloodtype  17 non-null     object 
 6   height     17 non-null     float64
 7   weight     17 non-null     float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.2+ KB


#### 테이블 내용 가져와 보기:

In [ ]:
# SQLite 데이터 베이스 생성(또는 연결) & Cursor 생성.
conn = sqlite3.connect("student.db")                         # 처음 실행시 신규 생성.
cur = conn.cursor()

In [ ]:
# SQL 테이블 생성.
sql = "CREATE TABLE IF NOT EXISTS studentlist ( name TEXT, gender TEXT, age INT, grade INT, absence TEXT, bloodtype TEXT, height REAL, weight REAL);"
cur.execute(sql);
conn.commit();

In [ ]:
# SQL 테이블 내용 클리어.
sql = "DELETE FROM studentlist;"
cur.execute(sql);
conn.commit();

In [ ]:
# SQL 테이블 내용 확인.
sql = "SELECT * FROM studentlist;"
cur.execute(sql);
res = cur.fetchall();
print(res)

[]


#### 한 행 씩 옮겨 담는다:

In [ ]:
# "studentlist" 테이블에 행 추가.
sql = "INSERT INTO studentlist VALUES ( ?, ?, ?, ?, ?, ?, ?, ? );"      # ?를 placeholder로 사용!
for i in range(nrows):
    x = df.loc[i].values                         # 삽입될 내용.
    x = list(x)
    x[2] = int(x[2])                               # INT64를 INT로 변환.
    x[3] = int(x[3])                               # INT64를 INT로 변환.
    cur.execute(sql,x);                            # 삽입.  
    conn.commit();

In [ ]:
# 테이블 내용 보기.
sql = "SELECT * FROM studentlist"
cur.execute(sql);
res = cur.fetchall()
for x in res:
    print(x)

('Jared Diamond', 'M', 23, 3, 'Y', 'O', 165.3, 68.2)
("Sarah O'Donnel", 'F', 22, 2, 'N', 'AB', 170.1, 53.0)
('Brian Martin', 'M', 24, 4, 'N', 'B', 175.0, 80.1)
('David Hassel', 'M', 23, 3, 'N', 'AB', 182.1, 85.7)
('Clara Rodriquez', 'F', 20, 1, 'Y', 'A', 168.0, 49.5)
('Jennifer Lorentz', 'F', 21, 2, 'N', 'O', 162.0, 52.0)
('Susan Clark', 'F', 22, 1, 'N', 'O', 155.2, 45.3)
('Margareth Jones', 'F', 23, 1, 'N', 'A', 176.9, 55.0)
('John Bertsch', 'M', 23, 3, 'N', 'B', 178.5, 64.2)
('Jake Timmerman', 'M', 22, 2, 'N', 'B', 176.1, 61.3)
('Joshua Connor', 'M', 24, 4, 'Y', 'O', 167.1, 62.0)
('John Matsuda', 'M', 22, 2, 'N', 'AB', 180.0, 75.8)
('Eddy Johnson', 'M', 21, 1, 'N', 'A', 162.2, 55.3)
('Rebecah Anderson', 'F', 23, 3, 'N', 'O', 176.1, 53.1)
('Linda Carter', 'F', 22, 2, 'N', 'B', 158.2, 45.2)
('Richard Swayze', 'M', 24, 4, 'Y', 'B', 168.6, 70.2)
('Andrew Daley', 'M', 21, 1, 'N', 'A', 169.2, 62.2)


#### 2). 데이터베이스의 내용을 데이터프레임으로 옮겨본다: 

In [ ]:
# 테이블 내용 가져온다.
sql = "SELECT * FROM studentlist"
cur.execute(sql);
res = cur.fetchall()

In [ ]:
res

[('Jared Diamond', 'M', 23, 3, 'Y', 'O', 165.3, 68.2),
 ("Sarah O'Donnel", 'F', 22, 2, 'N', 'AB', 170.1, 53.0),
 ('Brian Martin', 'M', 24, 4, 'N', 'B', 175.0, 80.1),
 ('David Hassel', 'M', 23, 3, 'N', 'AB', 182.1, 85.7),
 ('Clara Rodriquez', 'F', 20, 1, 'Y', 'A', 168.0, 49.5),
 ('Jennifer Lorentz', 'F', 21, 2, 'N', 'O', 162.0, 52.0),
 ('Susan Clark', 'F', 22, 1, 'N', 'O', 155.2, 45.3),
 ('Margareth Jones', 'F', 23, 1, 'N', 'A', 176.9, 55.0),
 ('John Bertsch', 'M', 23, 3, 'N', 'B', 178.5, 64.2),
 ('Jake Timmerman', 'M', 22, 2, 'N', 'B', 176.1, 61.3),
 ('Joshua Connor', 'M', 24, 4, 'Y', 'O', 167.1, 62.0),
 ('John Matsuda', 'M', 22, 2, 'N', 'AB', 180.0, 75.8),
 ('Eddy Johnson', 'M', 21, 1, 'N', 'A', 162.2, 55.3),
 ('Rebecah Anderson', 'F', 23, 3, 'N', 'O', 176.1, 53.1),
 ('Linda Carter', 'F', 22, 2, 'N', 'B', 158.2, 45.2),
 ('Richard Swayze', 'M', 24, 4, 'Y', 'B', 168.6, 70.2),
 ('Andrew Daley', 'M', 21, 1, 'N', 'A', 169.2, 62.2)]

In [ ]:
# 새로운 데이터 프레임 생성 & 확인.
df_new = pd.DataFrame(data=res, columns=my_header, index=None)
df_new.head(5)

#### 3). 뒷 정리:

In [ ]:
# 테이블 삭제.
sql = "DROP TABLE studentlist;"
cur.execute(sql);
conn.commit();

In [ ]:
conn.close()

#2. 취업멘토링

면접 

# 3. 첫번째 시험